In [3]:
import sys
import os

# Añadir la ruta absoluta de src al sys.path
src_path = os.path.abspath('../src')
if src_path not in sys.path:
    sys.path.append(src_path)

# Verificar que la ruta se ha añadido correctamente
print("Rutas actuales en sys.path:", sys.path)

# Importar las librerías necesarias
from bot import TradingBot
from data.data_loader import load_data
from strategies.base_strategy import MovingAverageStrategy
import pandas as pd
import numpy as np

# Cargar configuración y datos de mercado
bot = TradingBot(config_path='../config.json')
bot.load_market_data(file_path='../path/to/your/market_data.csv')
bot.run()

Rutas actuales en sys.path: ['/Library/Frameworks/Python.framework/Versions/3.12/lib/python312.zip', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/lib-dynload', '', '/Users/daniel/Desktop/bot/.venv/lib/python3.12/site-packages', '../src', '/Users/daniel/Desktop/bot/trading-bot/src']


KeyError: 'short_window'